## 第一部分：了解 nn.Module的基本操作

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

In [2]:
model = models.resnet18()

print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### 打印出 model底下所有 parameters 的 name 以及對應的 shape 

In [3]:
for name, param in model.named_parameters():
    print(name, '\t', param.shape, '\t', param.requires_grad)
    print()

conv1.weight 	 torch.Size([64, 3, 7, 7]) 	 True

bn1.weight 	 torch.Size([64]) 	 True

bn1.bias 	 torch.Size([64]) 	 True

layer1.0.conv1.weight 	 torch.Size([64, 64, 3, 3]) 	 True

layer1.0.bn1.weight 	 torch.Size([64]) 	 True

layer1.0.bn1.bias 	 torch.Size([64]) 	 True

layer1.0.conv2.weight 	 torch.Size([64, 64, 3, 3]) 	 True

layer1.0.bn2.weight 	 torch.Size([64]) 	 True

layer1.0.bn2.bias 	 torch.Size([64]) 	 True

layer1.1.conv1.weight 	 torch.Size([64, 64, 3, 3]) 	 True

layer1.1.bn1.weight 	 torch.Size([64]) 	 True

layer1.1.bn1.bias 	 torch.Size([64]) 	 True

layer1.1.conv2.weight 	 torch.Size([64, 64, 3, 3]) 	 True

layer1.1.bn2.weight 	 torch.Size([64]) 	 True

layer1.1.bn2.bias 	 torch.Size([64]) 	 True

layer2.0.conv1.weight 	 torch.Size([128, 64, 3, 3]) 	 True

layer2.0.bn1.weight 	 torch.Size([128]) 	 True

layer2.0.bn1.bias 	 torch.Size([128]) 	 True

layer2.0.conv2.weight 	 torch.Size([128, 128, 3, 3]) 	 True

layer2.0.bn2.weight 	 torch.Size([128]) 	 True

layer2.0.b

### 為了使 forward propagation 加速 並降低 memory 使用量，請將所有 parameters 的requires_grad 關閉，關閉之後執行  forward propagation

In [4]:
input_ = torch.randn(1, 3, 128, 128)

In [5]:
for param in model.parameters():
    param.requires_grad_(False)

In [6]:
for param in model.parameters():
    print(param.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [7]:
output = model(input_)
print(output.shape)

torch.Size([1, 1000])


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


## 第二部分：依照指令，以較簡潔的方式搭建出模型

* input_shape = torch.Size([10, 12])
* 先經過一層 nn.Linear(12, 10)
* 經過10層 nn.Linear(10, 10)
* 最後經過 nn.Linear(10, 3) 輸出
* 每一個 nn.Linear過完後要先經過 nn.BatchNorm1d 才能到下一層，輸出層不用


In [8]:
input_ = torch.randn(10, 12)
## 示範
Linear = nn.Linear(in_features=12, out_features=10)
BN = nn.BatchNorm1d(num_features=10)

In [9]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # "自行填入"
        self.linear = nn.Linear(12, 10)
        self.bn = nn.BatchNorm1d(10)
        self.linears = [nn.Linear(10, 10) for _ in range(10)]
        self.bns = [nn.BatchNorm1d(10) for _ in range(10)]
        self.output = nn.Linear(10, 3)

    def forward(self, x):
        # "自行填入"
        x = self.bn(self.linear(x))
        for i in range(10):
            x = self.bns[i](self.linears[i](x))
            print(i, 'running!')
        x = self.output(x)
        return x

In [10]:
model = Model()

In [11]:
model

Model(
  (linear): Linear(in_features=12, out_features=10, bias=True)
  (bn): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (output): Linear(in_features=10, out_features=3, bias=True)
)

In [12]:
for name, param in model.named_parameters():
    print(name, '\t', param.shape)

linear.weight 	 torch.Size([10, 12])
linear.bias 	 torch.Size([10])
bn.weight 	 torch.Size([10])
bn.bias 	 torch.Size([10])
output.weight 	 torch.Size([3, 10])
output.bias 	 torch.Size([3])


In [13]:
input_ = torch.randn(10,12)
outupt = model(input_)

0 running!
1 running!
2 running!
3 running!
4 running!
5 running!
6 running!
7 running!
8 running!
9 running!


In [14]:
outupt

tensor([[-0.6996,  1.2916, -0.3363],
        [-0.7340,  1.1711, -0.2765],
        [-1.1778,  0.4626,  0.0299],
        [-0.0177,  0.5350, -0.4176],
        [ 0.3852, -1.3095, -0.2575],
        [-0.8096, -0.1171, -0.1944],
        [ 0.1331, -0.0067, -0.4225],
        [ 1.0866, -0.7085, -0.4295],
        [-0.3667, -0.5736, -0.2028],
        [-0.4805,  0.1350, -0.3578]], grad_fn=<AddmmBackward>)